<a href="https://colab.research.google.com/github/Sreemaee21/Learning/blob/main/updatedmulticlass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.3 MB/s eta 0:00:00


In [2]:
pip install torch torchvision

In [3]:
from datasets import load_dataset

dataset = load_dataset("ohsumed")

Generating train split:   0%|          | 0/54709 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/293855 [00:00<?, ? examples/s]

In [4]:
from sklearn import svm
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.svm import SVC

In [5]:
from tqdm.notebook import tqdm

In [6]:
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer


In [7]:
train_data = dataset['train']
test_data = dataset['test']


In [8]:
train_data.shape, test_data.shape


((54709, 8), (293855, 8))

In [9]:
type(train_data), type(test_data)


(datasets.arrow_dataset.Dataset, datasets.arrow_dataset.Dataset)

In [10]:
for example in train_data:
    print(example)
    break


{'seq_id': 1, 'medline_ui': 87049087, 'mesh_terms': "Allied Health Personnel/*; Electric Countershock/*; Emergencies; Emergency Medical Technicians/*; Human; Prognosis; Recurrence; Support, U.S. Gov't, P.H.S.; Time Factors; Transportation of Patients; Ventricular Fibrillation/*TH.", 'title': 'Refibrillation managed by EMT-Ds: incidence and outcome without paramedic back-up.', 'publication_type': 'JOURNAL ARTICLE.', 'abstract': 'Some patients converted from ventricular fibrillation to organized rhythms by defibrillation-trained ambulance technicians (EMT-Ds) will refibrillate before hospital arrival. The authors analyzed 271 cases of ventricular fibrillation managed by EMT-Ds working without paramedic back-up. Of 111 patients initially converted to organized rhythms, 19 (17%) refibrillated, 11 (58%) of whom were reconverted to perfusing rhythms, including nine of 11 (82%) who had spontaneous pulses prior to refibrillation. Among patients initially converted to organized rhythms, hospita

In [11]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import torch


In [12]:

from nltk.tokenize import word_tokenize
from nltk import pos_tag, ne_chunk
from sklearn.feature_extraction.text import CountVectorizer


In [13]:
nltk.download('punkt')
nltk.download('stopwords')




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
import nltk
from nltk.corpus import stopwords
import string
import re

# Define a function for text preprocessing
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove extra spaces
    text = ' '.join(text.split())

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stop words
    filtered_tokens = [token for token in tokens if token not in stop_words]

    return " ".join(filtered_tokens)
    # return text



# Sample code to process the first 50 entries from the 'train_data'
nltk.download('punkt')  # Download punkt tokenizer
stop_words = set(stopwords.words('english'))





for data in tqdm((train_data)):

    # Preprocess 'title', 'abstract', and 'mesh_terms' fields
    data['title'] = preprocess_text(data['title'])
    data['abstract'] = preprocess_text(data['abstract'])
    data['mesh_terms'] = preprocess_text(data['mesh_terms'])

    # # Print the preprocessed data
    # print("Entry:", idx + 1)
    # print("Title:", data['title'])
    # print("Abstract:", data['abstract'])
    # print("Mesh Terms:", data['mesh_terms'])
    # print()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


  0%|          | 0/54709 [00:00<?, ?it/s]

In [15]:
temp_data = data["abstract"]

In [16]:
temp_data

'recent study expression monocyte differentiation antigens mag hladr peripheral blood monocytes pbm led recognition resting activated monocyte phenotypes former identified expression large amounts mag ie mo small amounts hladr latter identified reverse studied phenotypes pbm synovial fluid monocytes sfm patients chronic inflammatory arthritis found pbm primarily resting sfm usually activated addition measured degree modulation mag hladr gammainterferon gammaifn patient pbm reacted pbm normal individuals ie mag decreased hladr increased exposure gammaifn however patient sfm hladr increase exposure gammaifn expression already maximal interestingly mag could still downregulated gammaifntreated sfm even expression began low level ie activated phenotype independent regulation mag hladr suggests macrophage activating factors gammaifn may responsible part activated phenotypes observed'

In [17]:
%%time
preprocess_text(temp_data)

CPU times: user 496 µs, sys: 13 µs, total: 509 µs
Wall time: 513 µs


'recent study expression monocyte differentiation antigens mag hladr peripheral blood monocytes pbm led recognition resting activated monocyte phenotypes former identified expression large amounts mag ie mo small amounts hladr latter identified reverse studied phenotypes pbm synovial fluid monocytes sfm patients chronic inflammatory arthritis found pbm primarily resting sfm usually activated addition measured degree modulation mag hladr gammainterferon gammaifn patient pbm reacted pbm normal individuals ie mag decreased hladr increased exposure gammaifn however patient sfm hladr increase exposure gammaifn expression already maximal interestingly mag could still downregulated gammaifntreated sfm even expression began low level ie activated phenotype independent regulation mag hladr suggests macrophage activating factors gammaifn may responsible part activated phenotypes observed'

In [44]:
from joblib import Parallel, delayed
def pow2(x): return x**2
Parallel(n_jobs=<num_jobs>, backend=<backend>)(delayed(pow2)(x) for x in range(10))

SyntaxError: ignored

In [19]:
for data in tqdm((test_data)):

    # Preprocess 'title', 'abstract', and 'mesh_terms' fields
    data['title'] = preprocess_text(data['title'])
    data['abstract'] = preprocess_text(data['abstract'])
    data['mesh_terms'] = preprocess_text(data['mesh_terms'])

    # # Print the preprocessed data

  0%|          | 0/293855 [00:00<?, ?it/s]

In [20]:


def extract_terms(text):
  disease_categories = set()
  categories = text.split('; ')
    # Check if any of the categories belong to the 'Disease' or 'Syndrome' semantic type
  for category in categories:
     # if '/D' in category or '/T' in category:
        disease_categories.add(category)
  return list(disease_categories)



In [21]:
for x in test_data:
  print(x['abstract'])
  break

Ribonuclease A was reacted with [1-13C,1,2-14C]acetaldehyde and sodium cyanoborohydride in the presence or absence of 0.2 M phosphate. After several hours of incubation at 4 degrees C (pH 7.4) stable acetaldehyde-RNase adducts were formed, and the extent of their formation was similar regardless of the presence of phosphate. Although the total amount of covalent binding was comparable in the absence or presence of phosphate, this active site ligand prevented the inhibition of enzymatic activity seen in its absence. This protective action of phosphate diminished with progressive ethylation of RNase, indicating that the reversible association of phosphate with the active site lysyl residue was overcome by the irreversible process of reductive ethylation. Modified RNase was analysed using 13C proton decoupled NMR spectroscopy. Peaks arising from the covalent binding of enriched acetaldehyde to free amino groups in the absence of phosphate were as follows: NH2-terminal alpha amino group, 4

In [22]:
# Create a TF-IDF vectorizer to convert text data into numerical representations
vectorizer = TfidfVectorizer()

In [23]:
%%time
# Convert the data and labels to PyTorch tensors and move them to the GPU
X_train_texts = [data['title'] + ' ' + data['abstract']  for data in train_data]
y_train = [extract_terms(data['mesh_terms']) for data in train_data]



CPU times: user 6.06 s, sys: 85.7 ms, total: 6.14 s
Wall time: 6.15 s


In [24]:
X_test_texts = [data['title'] + ' ' + data['abstract'] for data in test_data]
y_test = [extract_terms(data['mesh_terms']) for data in test_data]


In [25]:

# Split the data into training and testing sets
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_texts, y_train, test_size=0.2, random_state=42)


In [45]:
X_train_vectorized = vectorizer.fit_transform(X_train_texts[:1000])


In [46]:
X_test_vectorized = vectorizer.transform(X_test_texts[:1000])


In [ ]:
# sckit learn 1000 iterations
# shukd be able to run

# **Convert y to integers**

In [47]:
term_vocab = set()

for x in y_train:
  term_vocab.update((x))

len(term_vocab)

106799

In [48]:
term2int = {x:i for (i,x) in enumerate(term_vocab)}

In [49]:
y_train_int = []
for x in tqdm(y_train[:100]):
  # labels = [term2int[y] for y in x]
  labels = np.zeros(len(term2int))
  for y in x:
    labels[term2int[y]] = 1
  y_train_int.append(labels)

y_train_int = np.stack(y_train_int)

  0%|          | 0/100 [00:00<?, ?it/s]

In [50]:
X_train_vectorized.shape, y_train_int.shape

((1000, 10441), (100, 106799))

In [51]:
subclf = SVC()
clf = OneVsRestClassifier(estimator=subclf)
clf.fit(X_train_vectorized, y_train_int)

/usr/local/lib/python3.10/dist-packages/sklearn/multiclass.py:77: UserWarning: Label not 0 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/multiclass.py:77: UserWarning: Label not 1 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/multiclass.py:77: UserWarning: Label not 2 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/multiclass.py:77: UserWarning: Label not 3 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/multiclass.py:77: UserWarning: Label not 4 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/multiclass.py:77: UserWarning: Label not 5 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/multiclass.py:77: UserWarning: Label not 6 is present in all training examples.
  

ValueError: ignored

In [52]:
y_test_int = []
for x in tqdm(y_test[:100]):
    # labels = [term2int[y] for y in x]
    labels = np.zeros(len(term2int))
    for y in x:
        if y in term2int:
            labels[term2int[y]] = 1
    y_test_int.append(labels)

y_test_int = np.stack(y_test_int)

  0%|          | 0/100 [00:00<?, ?it/s]

In [53]:
from sklearn.metrics import f1_score

In [54]:
y_pred = clf.predict(X_test_vectorized)

AttributeError: ignored

In [55]:
f1_score(y_test_int, y_pred, average='micro')

0.17296222664015903

In [56]:
y_test_int.shape

(100, 106799)

In [57]:
classifier = LogisticRegression(multi_class='auto')

In [39]:
X_train_vectorized.shape

(100, 1298)

In [40]:
y_train_int.shape

(100, 106799)

In [43]:
classifier.fit(X_train_vectorized, y_train_int)

ValueError: ignored

In [ ]:
classifier = svm.SVC(decision_function_shape = 'ovo')


In [ ]:
len(y_train_int)